# Analisis Dominosmania

## Iniciamos la conexion a snowflake

In [1]:
from snowflake.snowpark import Session
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')

connection_parameters: dict[str, int | str] = {
    'user' : config.get('SNOWFLAKE', 'USER'),
    'password' : config.get('SNOWFLAKE', 'PASSWORD'),
    'account' : config.get('SNOWFLAKE', 'ACCOUNT'),
    'database' : config.get('SNOWFLAKE', 'DATABASE'),
    'warehouse' : config.get('SNOWFLAKE', 'WAREHOUSE'),
    'schema' : config.get('SNOWFLAKE', 'SCHEMA'),
    'role' : config.get('SNOWFLAKE', 'ROLE'),
}

session = Session.builder.configs(connection_parameters).create()

## Fechas y cupones de las Dominosmanias

In [5]:
import pandas as pd

dominosmanias = session.create_dataframe(pd.read_csv('./dominosmanias.csv'))

dominosmanias.show()

TypeError: create_dataframe() function only accepts data as a list, tuple or a pandas DataFrame.

## Analisis de recompra

### Clientes que entraron por Dominosmania (Nuevos clientes y recurrentes)

In [ ]:
from DP_transacciones_total import get_transacciones_dp_total
from general_primeras_compras import get_primeras_compras
from snowflake.snowpark.functions import lit

transacciones_dp = get_transacciones_dp_total(session)

clientes_nuevos = (
    get_primeras_compras(session, transacciones_dp)
    .join(dominosmanias, on = 'FECHA')
    .select(['EMAIL', 'NOMBRE'])
    .distinct()
    .with_column('TIPO', lit('NUEVO'))
)

clientes_existentes = (
    transacciones_dp
    .join(dominosmanias, on = 'FECHA')
    .join(clientes_nuevos, on = ['EMAIL', 'NOMBRE'], how = 'LEFTANTI')
    .select(['EMAIL', 'NOMBRE'])
    .distinct()
    .with_column('TIPO', lit('EXISTENTE'))
)

clientes_que_entraron_por_dominosmania = clientes_nuevos.union_all(clientes_existentes)

clientes_que_entraron_por_dominosmania.show()

NameError: name 'dominosmanias' is not defined

### Cuantos volvieron a comprar? Cual es su frecuencia?

In [6]:
from snowflake.snowpark.functions import max, col, count_distinct, date_add

transacciones_despues_de_dominosmania = (
    dominosmanias
    .group_by('NOMBRE')
    .agg(max('FECHA').alias('FECHA_FIN'))
    .join(clientes_que_entraron_por_dominosmania, on = 'NOMBRE')
    .join(transacciones_dp, on = 'EMAIL')
    .filter(col('FECHA') > col('FECHA_FIN'))
    .filter(col('FECHA') <= date_add(col('FECHA_FIN'), 180))
)

volumen_total = clientes_que_entraron_por_dominosmania.group_by(['NOMBRE', 'TIPO']).agg(count_distinct('EMAIL').alias('CLIENTES'))
volumen_con_recompra = transacciones_despues_de_dominosmania.group_by(['NOMBRE', 'TIPO']).agg(count_distinct('EMAIL').alias('CLIENTES_CON_RECOMPRA'), count_distinct('ORDER_ID').alias('TRANSACCIONES'))

volumen_recompra_y_frecuencia = (
    volumen_total
    .join(volumen_con_recompra, on = ['NOMBRE', 'TIPO'])
    .with_column('Porcentaje con recompra', col('CLIENTES_CON_RECOMPRA') / col('CLIENTES'))
    .with_column('Frecuencia', col('TRANSACCIONES') / col('CLIENTES_CON_RECOMPRA'))
)

volumen_recompra_y_frecuencia.to_pandas().to_csv('wea.csv')

NameError: name 'dominosmanias' is not defined

### Entendimiento de la composicion de ordenes de la promocionalidad (Dominosmania, otros cupones, full price)

In [ ]:
# (
#     session.table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3').filter(col('FCIDDESCUENTO').isin([]))
# )

descuentos = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACDESCUENTOS')
    .filter(
        col('FIIDDESCUENTOSRC').like(r'%2D50%') |
        col('FIIDDESCUENTOSRC').like(r'%D229%') |
        col('FIIDDESCUENTOSRC').like(r'%DS199%')
    )
)

descuentos.show()
# descuentos.to_pandas().to_csv('cupones.csv')

NameError: name 'session' is not defined

### Cuantos clientes se mantienen o se pierden?

In [8]:
from snowflake.snowpark.functions import col, substring, lower, concat
from snowflake.snowpark import Session, DataFrame   

def get_transacciones_cloud(session: Session) -> DataFrame:
    transacciones_cloud = (
        session.table('SEGMENT_EVENTS.DOMINOS_GOLO.VENTA_CLOUD')
        .filter(~col('STOREID').like('9%'))
        .filter(col('SOURCEORGANIZATIONURI').is_not_null())
        .filter(col('SOURCEORGANIZATIONURI').isin(['order.dominos.com', 'resp-order.dominos.com', 'iphone.dominos.mx', 'android.dominos.mx']))
        .with_column('FECHA', substring(col('STOREORDERID'), 1, 10))
        .with_column('EMAIL', lower(col('EMAIL')))
        .with_column('ORDER_ID', concat(col('FECHA'), col('STOREID'), col('STOREORDERID')))
        .with_column('VENTA', col('PAYMENTSAMOUNT') / 1.16)
        .select(['EMAIL', 'FECHA', 'VENTA', 'ORDER_ID', 'PHONE'])
    )

    return transacciones_cloud

get transacciones_cloud(session)

SyntaxError: invalid syntax (572068987.py, line 19)

### Recuperacion de clientes

In [9]:
from DP_transacciones_total import get_transacciones_dp_total
from general_rangos_de_fecha import get_rangos
from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.functions import col, count_distinct
from typing import Literal

def get_activos_en_serie_de_tiempo(session: Session, transacciones_total:DataFrame, serie_de_tiempo:list[Literal['ANIO_ALSEA', 'MES_ALSEA', 'SEM_ALSEA', 'FECHA']], intervalo_de_dias:int = 180) -> DataFrame:
    if ('EMAIL' not in transacciones_total.columns) and ('FECHA' not in transacciones_total.columns):
        raise KeyError('El DataFrame de transacciones debe contener las columnas "EMAIL" y "FECHA"')
    elif 'EMAIL' not in transacciones_total.columns:
        raise KeyError('El DataFrame de transacciones debe contener una columna "EMAIL"')
    elif 'FECHA' not in transacciones_total.columns:
        raise KeyError('El DataFrame de transacciones debe contener una columna "FECHA"')

    rangos = get_rangos(session, serie_de_tiempo, {'FECHA_FIN': 0, 'FECHA_INICIO': -intervalo_de_dias})

    res = (
        transacciones_total
        .join(rangos)
        .filter(col('FECHA_INICIO') <= col('FECHA'))
        .filter(col('FECHA') <= col('FECHA_FIN'))
        .group_by('ANIO_ALSEA', 'SEM_ALSEA')
        .agg(count_distinct('EMAIL').alias('ACTIVOS'))
    )

    return res


get_activos_en_serie_de_tiempo()

TypeError: get_activos_en_serie_de_tiempo() missing 3 required positional arguments: 'session', 'transacciones_total', and 'serie_de_tiempo'